In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [ ]:
df_predict = pd.read_csv("../input/demand-forecasting-kernels-only/test.csv")
df_train = pd.read_csv("../input/demand-forecasting-kernels-only/train.csv")

On remet en forme le dataframe de sorte à numéroter les jours au lieu d'utiliser la date exacte : 

In [ ]:
jours = [31,28,31,30,31,30,31,31,30,31,30,31]

def date_codage(d):
    annee = int(d[:4])
    mois = int(d[5:7])
    jour = int(d[8:10])
    res = 0
    for i in range(annee-2013):
        res += 365
    for i in range(mois-1):
        res += jours[i]
    res += jour
    if annee == 2016:
        if mois > 2:
            res += 1
    elif annee >= 2017:
        res += 1
    return res

In [ ]:
num_date_train = [ date_codage(d) for d in df_train["date"] ]
num_date_predict = [ date_codage(d) for d in df_predict["date"] ]

df_train.insert(4, "num_date", num_date_train)
df_predict.insert(4, "num_date", num_date_predict)

In [ ]:
test = [ [], [], [] ]

for k in range(913000):
    if  df_train["num_date"][k] >= 1462:
        test[0].append(df_train["store"][k])
        test[1].append(df_train["item"][k])
        test[2].append(df_train["num_date"][k])

In [ ]:
Y_train = df_train["sales"]

X_train = df_train.drop("date", axis = 1)
X_train = X_train.drop("sales", axis = 1)
X_predict = df_predict.drop("date", axis = 1)
X_predict = X_predict.drop("id", axis = 1)

In [ ]:
df_train.head()

In [ ]:
df_predict.head()

Pour rendre plus efficace l'entraînement, la prédiction et l'analyse de nos résultats, nous allons nous intéresser à la **demande du 1er article** : 

In [ ]:
df_train_one=df_train.head(5*365+1)#on a un dataframe d'entraînement de 2013 à 2017 sur la demande du premier produit
df_predict_one=df_predict.head(31+28+31)#on veut prédire la demande du 1er produit sur les trois premiers mois de 2018

In [ ]:
df_train_one.shape

In [ ]:
df_predict_one.shape

# Entraînement avec différentes méthodes avant la prédiction:

En effet, n'ayant aucun moyen de vérifier les ventes en 2018 on ne peut pas juger correctement de l'effficacité des méthodes de prédiction utilisées.

Nous allons donc séparer les données contenues dans le dataframe **df_train_one** pour réaliser un premier entraînement et un test pour lequel on peut vérifier la précision des résultats. On prendra un test sur les trois derniers mois de 2017.

--> On pourra donc ensuite utiliser la méthode la plus efficace pour réaliser la prédiction "à l'aveugle" sur les 3 premiers mois de 2018. 

In [ ]:
X=df_train_one["num_date"]

y=df_train_one["sales"]

X_train_one=X[:3*365+366+5*31+28+3*30]#on sépare ici le test et le train non pas de manière aléatoire mais chronologique

X_test_one=X[3*365+366+5*31+28+3*30:]#le test se fait sur les trois derniers mois de 2017 pour l'instant

y_train_one=y[:3*365+366+5*31+28+3*30]

y_test_one=y[3*365+366+5*31+28+3*30:]

In [ ]:
plt.plot(X,y)
plt.show()

In [ ]:
plt.plot(X_train_one,y_train_one)
plt.show()

# Utilisation de Holt-Winters

Utilisation de Holt-Winters grâce au tutoriel de "Data Heroes" : https://www.youtube.com/watch?v=mQ352TiPvBI

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

On effectue un lissage exponentiel:

In [ ]:
model = ExponentialSmoothing(endog = y_train_one,trend = "add",seasonal = "add",seasonal_periods = 7).fit()

In [ ]:
predictions = model.forecast(steps = 14)

In [ ]:
df_train_one['sales'][1500:3*365+366+5*31+28+3*30].plot(figsize = (12,7))
predictions.plot()

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(X[1500:1826],y[1500:1826],color = 'blue')
plt.show()

Avec ce modèle de prédictionde Holt-Winters on obtient une prédiction qui n'est pas journalière, il est donc moins aisé de comparer les résultats obtenus aux réelles ventes : 

In [ ]:
X.shape

Néanmoins on remarque quand même sur le graphe que la prédiction que la prédiction est trop périodique et répétitive. 
Elle ne semble donc pas indiquée pour des périodes de prédiction de 3 mois.

# Méthode Prophet
Dans cette partie nous allons utiliser la librairie Prophet en nous inspirant du notebook sur la consommation d'énergie de M. Yim : https://www.kaggle.com/pyim59/s-ries-temporelles

On va d'abord réimporter nos dataframe en utilisant le format de date supporté par python:

In [ ]:
from fbprophet import Prophet

In [ ]:
df_prophet = pd.read_csv("../input/demand-forecasting-kernels-only/train.csv")
df_prophet['date'] = pd.to_datetime(df_prophet['date'])

In [ ]:
df_prophet.head()

On supprime les colonnes inutiles, on prend les données du premier article et on adapte le dataframe au format exigé par Prophet:

In [ ]:
df_prophet=df_prophet.drop(['store'],axis=1)
df_prophet=df_prophet.drop(['item'],axis=1)
df_prophet=df_prophet.head(5*365+1)
df_prophet.columns = ['ds','y']
df_prophet.head()

Maintenant on sépare en train et test:

In [ ]:
df_prophet_train = df_prophet[df_prophet['ds'] < '2017-10-01'].copy()
df_prophet_test = df_prophet[df_prophet['ds'] >= '2017-10-01'].copy()

Phase d'apprentissage:

In [ ]:
prophet = Prophet()
prophet.fit(df_prophet_train)
prediction = prophet.predict(df_prophet_test)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(df_prophet_train['ds'] , df_prophet_train['y'], color = 'blue')
plt.plot(df_prophet_test['ds'] , df_prophet_test['y'], color = 'black', alpha = 0.5)
plt.plot(prediction['ds'] , prediction['yhat'], color = 'orange', alpha = 0.8)
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(df_prophet_train['ds'] , df_prophet_train['y'], color = 'blue')
plt.plot(df_prophet_test['ds'] , df_prophet_test['y'], color = 'black', alpha = 0.3)
plt.plot(prediction['ds'] , prediction['yhat_upper'], color = 'red', alpha = 0.5)
plt.plot(prediction['ds'] , prediction['yhat_lower'], color = 'green', alpha = 0.5)
plt.show()

Finalement la prédiction via la librairie Prophet suit bien la tendance globale sur les 3 mois prédits et les prédictions 'upper' et 'lower'encadrent bien les ventes réelles.

On peut ici essayer d'évaluer la précision de la méthode sur ce dataframe en calculant l'**erreur quadratique**:

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse1=sqrt(mean_squared_error(df_prophet_test['y'],prediction['yhat']))
print(rmse1)

# Prédiction avec un réseau de neurones LSTM (Long Short-Term Memory):

Utilisation d'un réseau de neurone. (Source, tutoriel de Nachiketa Hebbar : https://www.youtube.com/watch?v=S8tpSG6Q2H0 )

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
df_lstm = pd.read_csv("../input/demand-forecasting-kernels-only/train.csv",index_col='date',parse_dates=True)

In [ ]:
df_lstm=df_lstm.drop(['store'],axis=1)
df_lstm=df_lstm.drop(['item'],axis=1)
df_lstm=df_lstm.head(5*365+1)

In [ ]:
df_lstm.head()

In [ ]:
df_lstm_train = df_lstm[:1734]
df_lstm_test = df_lstm[1734:]

In [ ]:
df_lstm_train.plot()

In [ ]:
results = seasonal_decompose(df_lstm_train['sales'])
results.plot();

On utilise le MinMaxScaler() pour mettre les ventes sur une échelle de 0 à 1 pour faciliter l'apprentissage et améliorer les résultats:

In [ ]:
scaler.fit(df_lstm_train)
scaled_train = scaler.transform(df_lstm_train)
scaled_test = scaler.transform(df_lstm_test)

In [ ]:
scaled_train[:10]

In [ ]:
n_input = 1733
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

X,y = generator[0]
print(f'Ayant les données: \n{X.flatten()}')
print(f'On prédit ce y: \n {y}')

In [ ]:
X.shape

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit(generator,epochs=40)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
last_train_batch = scaled_train[-1733:]

In [ ]:
last_train_batch = last_train_batch.reshape((1, n_input, n_features))

In [ ]:
model.predict(last_train_batch)

In [ ]:
scaled_test[0]

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
batch_actuel = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(df_lstm_test)):
    
    # valeur de la prédiction pour le premier batch
    pred_actuelle = model.predict(batch_actuel)[0]
    
    # ajouter la prédiction dans le tableau
    test_predictions.append(pred_actuelle) 
    
    # utiliser la prédiction pour mettre à jour le batch et retirer la première valeur
    batch_actuel = np.append(batch_actuel[:,1:,:],[[pred_actuelle]],axis=1)

In [ ]:
test_predictions

In [ ]:
df_lstm_test.head()

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
true_predictions

In [ ]:
df_lstm_test.head()

In [ ]:
df_lstm_test['predictions']=true_predictions

In [ ]:
df_lstm_test.head(92)

In [ ]:
df_lstm_test.plot(figsize=(14,5))

In [ ]:
rmse2=sqrt(mean_squared_error(df_lstm_test['sales'],df_lstm_test['predictions']))
print(rmse2)

On a ici un problème car la méthode dans tous mes tests ne donne qu'une série de valeurs croissantes, on a donc une grande erreur quadratique.

Au final je pense qu'il y a un problème dans le réseau de neurones qui ne permet pas de prédire correctement les ventes sur les 3 mois désirés.